In [54]:
import pandas as pd
import category_encoders as ce
from sklearn.model_selection import train_test_split #Modelo de segmentación de test and train
from sklearn.linear_model import Lasso #Modelo implementado para hacer la predicción del ingreso de las peliculas

In [55]:
movies_ojb = pd.read_csv(r'C:\Scripts\Proyectos\Predicción_de_ingresos_peliculas\Data\movies_obj.csv')

categoricals = pd.read_csv(r'C:\Scripts\Proyectos\Predicción_de_ingresos_peliculas\Data\categoricals.csv')

X = pd.read_csv(r'C:\Scripts\Proyectos\Predicción_de_ingresos_peliculas\Data\X_opening.csv')

In [56]:
y = X['worldwide_gross']

X = X[['production_budget', 'title_year', 'aspect_ratio',
       'duration.1', 'cast_total_facebook_likes', 'budget', 'imdb_score']]

Se realiza la validación inicial de cuantos valores unicos hay por cada feature de tipo obj con el fin de identificar que problemas en el aumento de la dimensión podrian presentarse.

In [57]:
movies_ojb.apply(pd.Series.nunique).sort_values() 

color                2
content_rating      18
language            47
country             65
genres             914
actor_1_name      2097
director_name     2398
actor_2_name      3032
actor_3_name      3521
plot_keywords     4760
movie_title       4917
dtype: int64

Las features más informativas son las del casting. Si embargo haciendo un one-hot encoding de estas estaríamos aumentando la dimensión por 2000 y algo!!

In [58]:
#!pip install category_encoders

Posterior a esto y por medio de category_encoders se realizara una codificación one-hot de tal manera de que se pueda alimentar el modelo con una mayor cantidad de features y así mejorar su desempeño. Para esto se toman los datos referentes a al actor principal y al director de la pelicula.

In [59]:
categoricals = categoricals.drop('Unnamed: 0',axis=1)

categoricals.head(2)

,actor_1_name,director_name
0,CCH Pounder,James Cameron
1,Doug Walker,Doug Walker


In [60]:
categoricals = categoricals.reset_index(drop=True).fillna(0)

In [61]:
X_binenc = pd.concat([X,categoricals],axis=1)

In [62]:
enconder = ce.BinaryEncoder(cols = ['actor_1_name','director_name'])

In [63]:
enconder.fit_transform(X_binenc).shape # De este proceso se obtienen un total de 29 columnas en las cuales se tienen actores y directores

(4104, 29)

In [64]:
X_binenc = enconder.fit_transform(X_binenc)

Teniendo estos nuevos datos se procede a hacer una nueva prueba del modelo

In [65]:
Xb_train, Xb_test, y_train, y_test = train_test_split(X_binenc,y)


In [66]:
X_train, X_test = (Xb_train[X.columns],Xb_test[X.columns])


In [67]:
model_binenc = Lasso()
model = Lasso()

In [68]:
model_binenc.fit(Xb_train,y_train)
model.fit(X_train,y_train)

Lasso()

In [69]:
print(model_binenc.score(Xb_test,y_test))
print(model.score(X_test,y_test))

0.5464984316063557
0.5461320258886385


Se ve un aumento en el rendimiento de nuestro algoritmo pero no de forma significativa. Mantengamos entonces la dimensionalidad de nuestro espacio de features baja, y vamos a buscar modelos más complejos.